In [1]:
import json
import shutil
import os

In [2]:
def select_data(j, origin_image, target_image, origin_anno, target_anno, json_path):
    
    """
    j: origin json file
    origin_image: origin image dataset
    target_image: the target folder of image dataset
    origin_anno: the folder of origin mask images
    target_anno: the folder of target mask images
    json_path: the generated new json file
    """

    annotations = j['annotations']
    annot_list = []
    image_ids = []
    i = 0

    for anno in annotations:

        seg_list = []
        segments = anno['segments_info']
        win_count = 0
        for seg in segments:
            category_id = seg['category_id']
            if category_id in [180, 181]:
                win_count = win_count + 1
            if category_id in [180, 181, 171, 175, 176, 177, 199]:
                seg_list.append(seg)

        if win_count != 0:
            new_annotation = {}
            new_annotation['segments_info'] = seg_list
            new_annotation['file_name'] = anno['file_name']
            new_annotation['image_id'] = anno['image_id']
            new_image_name = anno['file_name'].replace('png', 'jpg')
            annot_list.append(new_annotation)
            image_ids.append(anno['image_id'])
            shutil.copy(os.path.join(origin_image, new_image_name), target_image)
            shutil.copy(os.path.join(origin_anno, anno['file_name']), target_anno)

    new_image_list = []
    for image in j['images']:
        ID = image['id']
        if ID in image_ids:
            new_image_list.append(image)

    j['images'] = new_image_list
    j['annotations'] = annot_list

    #store josn
    
    with open(json_path, 'w') as json_file:
        json.dump(j, json_file)

In [3]:
path = r'/home/huihai/Downloads/panoptic_annotations_trainval2017/annotations/panoptic_val2017.json'
j = json.loads(open(path).read())

origin_image = r'/home/huihai/Huihai/Mask/Mask_RCNN/datasets/coco/val2017'
target_image = r'/home/huihai/Huihai/Mask/Mask_RCNN/datasets/coco/2017_wall_window/validation/image'
origin_anno = r'/home/huihai/Downloads/panoptic_annotations_trainval2017/annotations/panoptic_val2017/panoptic_val2017'
target_anno = r'/home/huihai/Huihai/Mask/Mask_RCNN/datasets/coco/2017_wall_window/validation/annotation_image'
json_path = r'/home/huihai/Huihai/Mask/Mask_RCNN/datasets/coco/2017_wall_window/validation/validation.json'
select_data(j, origin_image, target_image, origin_anno, target_anno, json_path)